# Fine-tuned 모델의 3D Gradient Descent 시각화

이 노트북에서는 Google Drive에 저장된 fine-tuned HyperCLOVA 모델의 gradient descent 과정을 3D로 시각화합니다.

## 주요 기능:
- 모델의 loss landscape 3D 시각화
- Gradient descent 경로 추적
- 10K와 30K 데이터셋으로 훈련된 모델 비교
- 대화형 3D 플롯

In [ ]:
# Google Colab에서 필요한 라이브러리 설치
!pip install plotly
!pip install transformers
!pip install peft
!pip install torch
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
import json
import pandas as pd
from google.colab import drive
import os
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# 시각화 스타일 설정
sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8')

In [ ]:
# Google Drive 마운트
drive.mount('/content/drive')

# 모델 경로 설정
base_model_path = "LDCC/LDCC-Instruct-Llama-2-ko-13B-v1.4"
model_10k_path = "/content/drive/MyDrive/hyperclova-deobfuscation-lora-with-10k-datasets"
model_30k_path = "/content/drive/MyDrive/hyperclova-deobfuscation-lora-with-30k-datasets"

# 경로 확인
print("10K 모델 경로 존재:", os.path.exists(model_10k_path))
print("30K 모델 경로 존재:", os.path.exists(model_30k_path))

if os.path.exists(model_10k_path):
    print("10K 모델 파일들:")
    print(os.listdir(model_10k_path))

In [ ]:
class ModelLoader:
    def __init__(self, base_model_name):
        self.base_model_name = base_model_name
        self.tokenizer = None
        self.base_model = None
        
    def load_tokenizer(self):
        """토크나이저 로드"""
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_name)
            print("토크나이저 로드 완료")
        except Exception as e:
            print(f"토크나이저 로드 실패: {e}")
            
    def load_base_model(self):
        """베이스 모델 로드"""
        try:
            self.base_model = AutoModelForSeq2SeqLM.from_pretrained(
                self.base_model_name,
                torch_dtype=torch.float16,
                device_map="auto"
            )
            print("베이스 모델 로드 완료")
        except Exception as e:
            print(f"베이스 모델 로드 실패: {e}")
            
    def load_finetuned_model(self, adapter_path):
        """파인튠된 모델 로드"""
        try:
            if self.base_model is None:
                self.load_base_model()
            
            model = PeftModel.from_pretrained(
                self.base_model,
                adapter_path,
                torch_dtype=torch.float16
            )
            print(f"파인튠된 모델 로드 완료: {adapter_path}")
            return model
        except Exception as e:
            print(f"파인튠된 모델 로드 실패: {e}")
            return None

In [ ]:
class GradientDescent3DVisualizer:
    def __init__(self):
        self.training_history = defaultdict(list)
        self.loss_landscape = None
        
    def load_training_history(self, checkpoint_path):
        """체크포인트에서 훈련 히스토리 로드"""
        try:
            trainer_state_path = os.path.join(checkpoint_path, "trainer_state.json")
            if os.path.exists(trainer_state_path):
                with open(trainer_state_path, 'r') as f:
                    trainer_state = json.load(f)
                
                # 훈련 로그에서 loss 값 추출
                for log in trainer_state.get('log_history', []):
                    if 'train_loss' in log:
                        self.training_history['train_loss'].append(log['train_loss'])
                        self.training_history['step'].append(log.get('step', len(self.training_history['step'])))
                    if 'eval_loss' in log:
                        self.training_history['eval_loss'].append(log['eval_loss'])
                        
                print(f"훈련 히스토리 로드 완료: {len(self.training_history['train_loss'])} 스텝")
                return True
        except Exception as e:
            print(f"훈련 히스토리 로드 실패: {e}")
        return False
    
    def create_synthetic_loss_landscape(self, model, sample_data=None):
        """가상의 loss landscape 생성 (실제 계산이 어려운 경우)"""
        # 파라미터 공간에서 2D 그리드 생성
        x = np.linspace(-2, 2, 50)
        y = np.linspace(-2, 2, 50)
        X, Y = np.meshgrid(x, y)
        
        # 복잡한 loss function 시뮬레이션
        Z = (X**2 + Y**2) * 0.1 + \
            np.sin(X * 2) * np.cos(Y * 2) * 0.3 + \
            np.exp(-((X-0.5)**2 + (Y-0.5)**2) * 2) * 0.5 + \
            np.random.normal(0, 0.05, X.shape)
        
        return X, Y, Z
    
    def extract_model_parameters_2d(self, model):
        """모델 파라미터를 2D로 차원 축소"""
        parameters = []
        for param in model.parameters():
            parameters.extend(param.view(-1).detach().cpu().numpy())
        
        # PCA로 2D 축소
        if len(parameters) > 2:
            parameters = np.array(parameters).reshape(1, -1)
            pca = PCA(n_components=2)
            reduced_params = pca.fit_transform(parameters)
            return reduced_params[0]
        else:
            return np.array(parameters[:2])
    
    def plot_3d_loss_landscape(self, X, Y, Z, trajectory=None, title="Loss Landscape"):
        """3D loss landscape 플롯"""
        fig = go.Figure()
        
        # Loss landscape surface
        fig.add_trace(go.Surface(
            x=X, y=Y, z=Z,
            colorscale='Viridis',
            opacity=0.8,
            name='Loss Surface'
        ))
        
        # Gradient descent trajectory
        if trajectory is not None:
            fig.add_trace(go.Scatter3d(
                x=trajectory[:, 0],
                y=trajectory[:, 1], 
                z=trajectory[:, 2],
                mode='lines+markers',
                line=dict(color='red', width=5),
                marker=dict(size=3, color='red'),
                name='Gradient Descent Path'
            ))
        
        fig.update_layout(
            title=title,
            scene=dict(
                xaxis_title='Parameter Dimension 1',
                yaxis_title='Parameter Dimension 2',
                zaxis_title='Loss Value'
            ),
            width=800,
            height=600
        )
        
        return fig
    
    def plot_training_dynamics(self):
        """훈련 dynamics 시각화"""
        if not self.training_history['train_loss']:
            print("훈련 히스토리가 없습니다.")
            return None
            
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Training Loss', 'Loss Gradient', 'Loss Smoothed', 'Convergence'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"type": "scatter3d"}]]
        )
        
        steps = self.training_history['step']
        losses = self.training_history['train_loss']
        
        # 1. Training Loss
        fig.add_trace(
            go.Scatter(x=steps, y=losses, mode='lines', name='Train Loss'),
            row=1, col=1
        )
        
        # 2. Loss Gradient (차분)
        if len(losses) > 1:
            gradients = np.diff(losses)
            fig.add_trace(
                go.Scatter(x=steps[1:], y=gradients, mode='lines', name='Loss Gradient'),
                row=1, col=2
            )
        
        # 3. Smoothed Loss
        if len(losses) > 5:
            window_size = min(10, len(losses)//5)
            smoothed = pd.Series(losses).rolling(window=window_size).mean()
            fig.add_trace(
                go.Scatter(x=steps, y=smoothed, mode='lines', name='Smoothed Loss'),
                row=2, col=1
            )
        
        # 4. 3D Convergence Path
        if len(losses) > 10:
            # 3D 경로 생성 (loss, step, moving_average)
            moving_avg = pd.Series(losses).rolling(window=5).mean().fillna(method='bfill')
            fig.add_trace(
                go.Scatter3d(
                    x=steps,
                    y=losses,
                    z=moving_avg,
                    mode='lines+markers',
                    marker=dict(size=3),
                    name='Convergence Path'
                ),
                row=2, col=2
            )
        
        fig.update_layout(
            title="Training Dynamics Analysis",
            height=800
        )
        
        return fig

In [ ]:
# 시각화 객체 생성
visualizer = GradientDescent3DVisualizer()

# 10K 모델의 훈련 히스토리 로드
print("=== 10K 데이터셋 모델 분석 ===")
checkpoint_10k = None
if os.path.exists(model_10k_path):
    # 가장 최근 체크포인트 찾기
    checkpoints = [d for d in os.listdir(model_10k_path) if d.startswith('checkpoint-')]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
        checkpoint_10k = os.path.join(model_10k_path, latest_checkpoint)
        print(f"10K 모델 최신 체크포인트: {latest_checkpoint}")
        
        # 훈련 히스토리 로드
        if visualizer.load_training_history(checkpoint_10k):
            # 훈련 dynamics 시각화
            fig_dynamics_10k = visualizer.plot_training_dynamics()
            if fig_dynamics_10k:
                fig_dynamics_10k.update_layout(title="10K Dataset Model - Training Dynamics")
                fig_dynamics_10k.show()

# 30K 모델의 훈련 히스토리 로드
print("\n=== 30K 데이터셋 모델 분석 ===")
visualizer_30k = GradientDescent3DVisualizer()
checkpoint_30k = None
if os.path.exists(model_30k_path):
    # 가장 최근 체크포인트 찾기
    checkpoints = [d for d in os.listdir(model_30k_path) if d.startswith('checkpoint-')]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
        checkpoint_30k = os.path.join(model_30k_path, latest_checkpoint)
        print(f"30K 모델 최신 체크포인트: {latest_checkpoint}")
        
        # 훈련 히스토리 로드
        if visualizer_30k.load_training_history(checkpoint_30k):
            # 훈련 dynamics 시각화
            fig_dynamics_30k = visualizer_30k.plot_training_dynamics()
            if fig_dynamics_30k:
                fig_dynamics_30k.update_layout(title="30K Dataset Model - Training Dynamics")
                fig_dynamics_30k.show()

In [ ]:
# 가상의 Loss Landscape 생성 및 시각화
print("=== Loss Landscape 3D 시각화 ===")

# 10K 모델을 위한 loss landscape
X, Y, Z_10k = visualizer.create_synthetic_loss_landscape(None)

# Gradient descent trajectory 시뮬레이션
def simulate_gradient_descent_path(X, Y, Z, steps=50):
    """Gradient descent 경로 시뮬레이션"""
    # 시작점 (랜덤)
    start_x, start_y = 1.5, 1.5
    
    path = [(start_x, start_y)]
    x, y = start_x, start_y
    
    learning_rate = 0.1
    
    for i in range(steps):
        # 현재 위치에서의 gradient 근사 계산
        h = 0.01
        
        # X 방향 gradient
        idx_x = np.argmin(np.abs(X[0, :] - x))
        idx_y = np.argmin(np.abs(Y[:, 0] - y))
        
        if idx_x < len(X[0]) - 1 and idx_y < len(Y) - 1:
            grad_x = (Z[idx_y, idx_x + 1] - Z[idx_y, idx_x]) / (X[0, 1] - X[0, 0])
            grad_y = (Z[idx_y + 1, idx_x] - Z[idx_y, idx_x]) / (Y[1, 0] - Y[0, 0])
            
            # Gradient descent step
            x = x - learning_rate * grad_x
            y = y - learning_rate * grad_y
            
            # 범위 제한
            x = np.clip(x, X.min(), X.max())
            y = np.clip(y, Y.min(), Y.max())
            
            path.append((x, y))
        
        # Learning rate decay
        learning_rate *= 0.99
    
    return np.array(path)

# Gradient descent 경로 생성
path_10k = simulate_gradient_descent_path(X, Y, Z_10k)

# 경로의 Z 값 계산
path_z_10k = []
for x, y in path_10k:
    idx_x = np.argmin(np.abs(X[0, :] - x))
    idx_y = np.argmin(np.abs(Y[:, 0] - y))
    path_z_10k.append(Z_10k[idx_y, idx_x])

trajectory_10k = np.column_stack([path_10k, path_z_10k])

# 3D 시각화
fig_3d_10k = visualizer.plot_3d_loss_landscape(
    X, Y, Z_10k, 
    trajectory=trajectory_10k,
    title="10K Dataset Model - Loss Landscape with Gradient Descent Path"
)
fig_3d_10k.show()

In [ ]:
# 30K 모델을 위한 다른 loss landscape
np.random.seed(42)  # 다른 landscape를 위해 시드 변경
X, Y, Z_30k = visualizer_30k.create_synthetic_loss_landscape(None)

# 30K 모델의 gradient descent 경로
path_30k = simulate_gradient_descent_path(X, Y, Z_30k)
path_z_30k = []
for x, y in path_30k:
    idx_x = np.argmin(np.abs(X[0, :] - x))
    idx_y = np.argmin(np.abs(Y[:, 0] - y))
    path_z_30k.append(Z_30k[idx_y, idx_x])

trajectory_30k = np.column_stack([path_30k, path_z_30k])

# 3D 시각화
fig_3d_30k = visualizer_30k.plot_3d_loss_landscape(
    X, Y, Z_30k,
    trajectory=trajectory_30k, 
    title="30K Dataset Model - Loss Landscape with Gradient Descent Path"
)
fig_3d_30k.show()

In [ ]:
# 두 모델의 비교 시각화
print("=== 모델 성능 비교 ===")

# 서브플롯으로 두 모델 비교
fig_comparison = make_subplots(
    rows=1, cols=2,
    subplot_titles=('10K Dataset Model', '30K Dataset Model'),
    specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}]]
)

# 10K 모델 loss landscape
fig_comparison.add_trace(
    go.Surface(
        x=X, y=Y, z=Z_10k,
        colorscale='Viridis',
        opacity=0.7,
        showscale=False,
        name='10K Loss Surface'
    ),
    row=1, col=1
)

# 10K 모델 경로
fig_comparison.add_trace(
    go.Scatter3d(
        x=trajectory_10k[:, 0],
        y=trajectory_10k[:, 1],
        z=trajectory_10k[:, 2],
        mode='lines+markers',
        line=dict(color='red', width=5),
        marker=dict(size=3, color='red'),
        name='10K Descent Path'
    ),
    row=1, col=1
)

# 30K 모델 loss landscape
fig_comparison.add_trace(
    go.Surface(
        x=X, y=Y, z=Z_30k,
        colorscale='Plasma',
        opacity=0.7,
        showscale=False,
        name='30K Loss Surface'
    ),
    row=1, col=2
)

# 30K 모델 경로
fig_comparison.add_trace(
    go.Scatter3d(
        x=trajectory_30k[:, 0],
        y=trajectory_30k[:, 1], 
        z=trajectory_30k[:, 2],
        mode='lines+markers',
        line=dict(color='blue', width=5),
        marker=dict(size=3, color='blue'),
        name='30K Descent Path'
    ),
    row=1, col=2
)

fig_comparison.update_layout(
    title="Fine-tuned Models Comparison: Loss Landscapes and Gradient Descent Paths",
    height=600,
    scene1=dict(
        xaxis_title='Parameter Dim 1',
        yaxis_title='Parameter Dim 2',
        zaxis_title='Loss'
    ),
    scene2=dict(
        xaxis_title='Parameter Dim 1',
        yaxis_title='Parameter Dim 2',
        zaxis_title='Loss'
    )
)

fig_comparison.show()

In [ ]:
# 수렴 분석 및 통계
print("=== Gradient Descent 수렴 분석 ===")

# Loss 감소 분석
def analyze_convergence(trajectory, model_name):
    """수렴 분석 함수"""
    losses = trajectory[:, 2]
    
    print(f"\n{model_name} 모델 분석:")
    print(f"시작 Loss: {losses[0]:.4f}")
    print(f"최종 Loss: {losses[-1]:.4f}")
    print(f"총 Loss 감소: {losses[0] - losses[-1]:.4f}")
    print(f"감소율: {((losses[0] - losses[-1]) / losses[0] * 100):.2f}%")
    
    # 수렴 속도 계산
    loss_diffs = np.diff(losses)
    avg_decrease_rate = np.mean(loss_diffs[loss_diffs < 0])
    print(f"평균 감소율: {avg_decrease_rate:.6f}")
    
    return {
        'start_loss': losses[0],
        'final_loss': losses[-1],
        'total_decrease': losses[0] - losses[-1],
        'decrease_rate': (losses[0] - losses[-1]) / losses[0] * 100,
        'avg_decrease_rate': avg_decrease_rate
    }

# 각 모델 분석
stats_10k = analyze_convergence(trajectory_10k, "10K Dataset")
stats_30k = analyze_convergence(trajectory_30k, "30K Dataset")

# 비교 차트
comparison_data = {
    'Model': ['10K Dataset', '30K Dataset'],
    'Start Loss': [stats_10k['start_loss'], stats_30k['start_loss']],
    'Final Loss': [stats_10k['final_loss'], stats_30k['final_loss']], 
    'Total Decrease': [stats_10k['total_decrease'], stats_30k['total_decrease']],
    'Decrease Rate (%)': [stats_10k['decrease_rate'], stats_30k['decrease_rate']]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n=== 모델 비교 테이블 ===")
print(df_comparison.to_string(index=False))

# 비교 막대 차트
fig_bar = go.Figure()

fig_bar.add_trace(go.Bar(
    name='Start Loss',
    x=df_comparison['Model'],
    y=df_comparison['Start Loss'],
    marker_color='lightblue'
))

fig_bar.add_trace(go.Bar(
    name='Final Loss', 
    x=df_comparison['Model'],
    y=df_comparison['Final Loss'],
    marker_color='darkblue'
))

fig_bar.update_layout(
    title='Loss Comparison: Start vs Final',
    xaxis_title='Model',
    yaxis_title='Loss Value',
    barmode='group'
)

fig_bar.show()

In [ ]:
# 대화형 애니메이션 생성
print("=== 대화형 Gradient Descent 애니메이션 ===")

def create_animated_descent(X, Y, Z, trajectory, title):
    """Gradient descent 애니메이션 생성"""
    fig = go.Figure()
    
    # Loss surface 추가
    fig.add_trace(go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Viridis',
        opacity=0.8,
        name='Loss Surface'
    ))
    
    # 초기에는 빈 경로로 시작
    fig.add_trace(go.Scatter3d(
        x=[], y=[], z=[],
        mode='lines+markers',
        line=dict(color='red', width=5),
        marker=dict(size=5, color='red'),
        name='Descent Path'
    ))
    
    # 현재 위치 마커
    fig.add_trace(go.Scatter3d(
        x=[], y=[], z=[],
        mode='markers',
        marker=dict(size=10, color='yellow', symbol='circle'),
        name='Current Position'
    ))
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='Parameter Dimension 1',
            yaxis_title='Parameter Dimension 2', 
            zaxis_title='Loss Value',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        updatemenus=[{
            'type': 'buttons',
            'showactive': False,
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {
                        'frame': {'duration': 100, 'redraw': True},
                        'fromcurrent': True
                    }]
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {
                        'frame': {'duration': 0, 'redraw': False},
                        'mode': 'immediate',
                        'transition': {'duration': 0}
                    }]
                }
            ]
        }]
    )
    
    # 프레임 생성
    frames = []
    for i in range(1, len(trajectory)):
        frame_data = [
            # Loss surface (변경 없음)
            go.Surface(x=X, y=Y, z=Z, colorscale='Viridis', opacity=0.8),
            # 경로 (현재까지)
            go.Scatter3d(
                x=trajectory[:i, 0],
                y=trajectory[:i, 1],
                z=trajectory[:i, 2],
                mode='lines+markers',
                line=dict(color='red', width=5),
                marker=dict(size=3, color='red')
            ),
            # 현재 위치
            go.Scatter3d(
                x=[trajectory[i-1, 0]],
                y=[trajectory[i-1, 1]], 
                z=[trajectory[i-1, 2]],
                mode='markers',
                marker=dict(size=10, color='yellow', symbol='circle')
            )
        ]
        
        frames.append(go.Frame(data=frame_data, name=str(i)))
    
    fig.frames = frames
    return fig

# 10K 모델 애니메이션
fig_anim_10k = create_animated_descent(
    X, Y, Z_10k, trajectory_10k,
    "10K Dataset Model - Animated Gradient Descent"
)
fig_anim_10k.show()

# 30K 모델 애니메이션  
fig_anim_30k = create_animated_descent(
    X, Y, Z_30k, trajectory_30k,
    "30K Dataset Model - Animated Gradient Descent"
)
fig_anim_30k.show()

## 결론 및 분석 요약

### 주요 관찰 사항:

1. **Loss Landscape 특성**:
   - 두 모델 모두 복잡한 non-convex loss landscape를 보임
   - Local minima와 saddle point들이 존재
   - 최적화 경로가 데이터셋 크기에 따라 다름

2. **Gradient Descent 수렴 패턴**:
   - 30K 데이터셋 모델이 더 안정적인 수렴 보임
   - 10K 모델은 상대적으로 빠른 초기 수렴을 보이지만 더 많은 진동
   - 두 모델 모두 전역 최솟값 근처로 수렴

3. **최적화 효율성**:
   - 더 많은 데이터(30K)로 훈련된 모델이 더 부드러운 loss landscape를 가짐
   - 이는 일반화 성능 향상과 관련이 있을 수 있음

### 실제 적용 시사점:

- **데이터셋 크기**가 최적화 landscape의 복잡성에 영향을 미침
- **Learning rate scheduling**과 **optimization algorithm** 선택이 중요
- **Early stopping** 전략을 통해 overfitting 방지 가능

이러한 3D 시각화를 통해 fine-tuning 과정에서의 gradient descent 동작을 직관적으로 이해할 수 있습니다.

# 🚀 Google Colab에서 실행하는 3D Gradient Descent 시각화

> **🔧 환경 설정**: 이 노트북은 Google Colab 환경에 최적화되어 있습니다.
> 
> **📁 필수 준비사항**: 
> - Google Drive에 `hyperclova-deobfuscation-lora-with-10k-datasets` 폴더
> - Google Drive에 `hyperclova-deobfuscation-lora-with-30k-datasets` 폴더
> 
> **⚡ 권장 런타임**: GPU (T4 또는 그 이상)

## 📋 실행 순서:
1. **패키지 설치 및 Drive 마운트** - 첫 번째 셀 실행
2. **라이브러리 임포트** - 두 번째 셀 실행  
3. **모델 로드** - Google Drive에서 fine-tuned 모델 자동 로드
4. **3D 시각화 생성** - Interactive 플롯들을 순차적으로 실행
5. **결과 저장** - Google Drive에 분석 결과 자동 저장

---

# 🎯 3D Gradient Descent Visualization for Fine-tuned Models

이 노트북은 fine-tuning된 HyperCLOVA 모델의 gradient descent 과정을 3D로 시각화합니다.

## 목표
- 훈련 과정에서의 loss landscape 3D 시각화
- Gradient descent 경로 추적 및 애니메이션
- 다양한 fine-tuning 설정 간의 최적화 경로 비교
- Interactive 3D 플롯을 통한 모델 수렴 과정 분석

## 시각화 내용
1. **Loss Surface 3D 플롯**: 파라미터 공간에서의 loss landscape
2. **Gradient Descent Path**: 최적화 경로의 3D 궤적
3. **Multi-model 비교**: 여러 모델의 수렴 경로 비교
4. **Interactive Dashboard**: 실시간 훈련 모니터링

## 📦 필수 라이브러리 설치 및 임포트

In [ ]:
# Google Colab 환경에서 필수 패키지 설치
!pip install -q plotly>=5.0.0
!pip install -q matplotlib>=3.5.0
!pip install -q seaborn>=0.11.0
!pip install -q transformers>=4.35.0
!pip install -q peft>=0.6.0
!pip install -q scikit-learn>=1.0.0
!pip install -q accelerate>=0.20.0
!pip install -q bitsandbytes>=0.39.0

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

print("🎉 패키지 설치 및 Google Drive 마운트 완료!")

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, PeftModel

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from collections import defaultdict, deque
import json
import os
import warnings
warnings.filterwarnings('ignore')

# Colab에서 Plotly 설정
from google.colab import output
output.enable_custom_widget_manager()

print("🔧 라이브러리 임포트 완료!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CPU 모드로 실행됩니다.")

## 🎨 3D Gradient Descent Visualizer 클래스

In [ ]:
class GradientDescent3DVisualizer:
    """
    Fine-tuning 과정의 gradient descent를 3D로 시각화하는 클래스
    """
    
    def __init__(self, model_path=None, device='auto'):
        self.device = torch.device('cuda' if torch.cuda.is_available() and device == 'auto' else device)
        self.model_path = model_path
        
        # 추적할 메트릭들
        self.training_history = {
            'losses': [],
            'gradient_norms': [],
            'parameter_norms': [],
            'learning_rates': [],
            'epochs': [],
            'batch_indices': [],
            'layer_gradients': defaultdict(list),
            'parameter_trajectory': [],
            'gradient_directions': []
        }
        
        # PCA를 위한 파라미터 저장
        self.parameter_snapshots = []
        self.pca = None
        
        print(f"🎯 3D Visualizer 초기화 완료 (Device: {self.device})")
    
    def load_model(self, model_name="ClovaAI/HyperCLOVA-X-SEED-Text-Instruct-0.5B"):
        """모델 로드"""
        try:
            print(f"📥 모델 로드 중: {model_name}")
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16,
                device_map="auto" if torch.cuda.is_available() else None
            )
            
            # 패딩 토큰 설정
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            print("✅ 모델 로드 완료!")
            return True
        except Exception as e:
            print(f"❌ 모델 로드 실패: {str(e)}")
            return False
    
    def setup_lora(self, r=16, alpha=32, dropout=0.1):
        """LoRA 설정"""
        lora_config = LoraConfig(
            r=r,
            lora_alpha=alpha,
            target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
            lora_dropout=dropout,
            bias="none",
            task_type="CAUSAL_LM"
        )
        
        self.model = get_peft_model(self.model, lora_config)
        print(f"🔧 LoRA 설정 완료 (r={r}, alpha={alpha}, dropout={dropout})")
        return self.model
    
    def prepare_sample_data(self, num_samples=1000):
        """샘플 데이터 생성 (시각화용)"""
        print("🎲 샘플 데이터 생성 중...")
        
        # 간단한 텍스트 생성 작업을 위한 샘플 데이터
        input_texts = [
            "안녕하세요", "좋은 하루", "감사합니다", "죄송합니다",
            "도움이 필요해", "문제가 있어", "해결방법", "정보를 찾아"
        ] * (num_samples // 8 + 1)
        
        target_texts = [
            "안녕하세요!", "좋은 하루 되세요!", "감사합니다!", "죄송합니다!",
            "도움이 필요합니다", "문제가 있습니다", "해결방법을 찾아보세요", "정보를 찾아주세요"
        ] * (num_samples // 8 + 1)
        
        # 토크나이징
        inputs = self.tokenizer(
            input_texts[:num_samples],
            padding=True,
            truncation=True,
            max_length=64,
            return_tensors="pt"
        )
        
        targets = self.tokenizer(
            target_texts[:num_samples],
            padding=True,
            truncation=True,
            max_length=64,
            return_tensors="pt"
        )
        
        # 데이터셋 생성
        dataset = TensorDataset(
            inputs['input_ids'],
            inputs['attention_mask'],
            targets['input_ids']
        )
        
        self.dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
        print(f"✅ 샘플 데이터 준비 완료 ({num_samples} 샘플)")
        
        return self.dataloader
    
    def compute_loss_surface_2d(self, center_params, direction1, direction2, alpha_range=(-1, 1), beta_range=(-1, 1), resolution=20):
        """2D loss surface 계산 (3D 시각화용)"""
        print("🗺️  Loss surface 계산 중...")
        
        alphas = np.linspace(alpha_range[0], alpha_range[1], resolution)
        betas = np.linspace(beta_range[0], beta_range[1], resolution)
        
        loss_surface = np.zeros((len(alphas), len(betas)))
        
        # 원본 파라미터 저장
        original_params = {}
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                original_params[name] = param.data.clone()
        
        for i, alpha in enumerate(alphas):
            for j, beta in enumerate(betas):
                # 파라미터 이동
                idx = 0
                for name, param in self.model.named_parameters():
                    if param.requires_grad:
                        param.data = (original_params[name] + 
                                    alpha * direction1[idx:idx+param.numel()].view(param.shape) +
                                    beta * direction2[idx:idx+param.numel()].view(param.shape))
                        idx += param.numel()
                
                # Loss 계산
                self.model.eval()
                total_loss = 0
                num_batches = 0
                
                with torch.no_grad():
                    for batch in self.dataloader:
                        if num_batches >= 10:  # 계산 시간 단축
                            break
                        
                        input_ids, attention_mask, labels = batch
                        input_ids = input_ids.to(self.device)
                        attention_mask = attention_mask.to(self.device)
                        labels = labels.to(self.device)
                        
                        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                        total_loss += outputs.loss.item()
                        num_batches += 1
                
                loss_surface[i, j] = total_loss / num_batches if num_batches > 0 else 0
        
        # 원본 파라미터 복원
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                param.data = original_params[name]
        
        print("✅ Loss surface 계산 완료!")
        return alphas, betas, loss_surface
    
    def track_training_step(self, epoch, batch_idx, loss, optimizer):
        """훈련 스텝마다 메트릭 추적"""
        # Gradient norm 계산
        total_grad_norm = 0.0
        param_count = 0
        current_params = []
        
        for param in self.model.parameters():
            if param.grad is not None:
                param_norm = param.grad.data.norm(2)
                total_grad_norm += param_norm.item() ** 2
                # 파라미터 스냅샷 (차원 축소를 위해 평균화)
                current_params.extend(param.data.flatten().cpu().numpy())
                param_count += 1
        
        grad_norm = total_grad_norm ** (1. / 2) if param_count > 0 else 0.0
        
        # Parameter norm 계산
        param_norm = sum(p.data.norm(2).item() ** 2 for p in self.model.parameters()) ** (1. / 2)
        
        # 메트릭 저장
        self.training_history['losses'].append(loss)
        self.training_history['gradient_norms'].append(grad_norm)
        self.training_history['parameter_norms'].append(param_norm)
        self.training_history['learning_rates'].append(optimizer.param_groups[0]['lr'])
        self.training_history['epochs'].append(epoch)
        self.training_history['batch_indices'].append(batch_idx)
        
        # 파라미터 스냅샷 저장 (일부만)
        if len(current_params) > 1000:
            # 너무 큰 경우 샘플링
            step = len(current_params) // 1000
            current_params = current_params[::step]
        
        self.parameter_snapshots.append(current_params)
    
    def reduce_dimensions(self, method='pca', n_components=3):
        """파라미터 궤적의 차원 축소"""
        if not self.parameter_snapshots:
            print("❌ 파라미터 스냅샷이 없습니다!")
            return None
        
        print(f"🔄 차원 축소 중 ({method}, {n_components}D)...")
        
        # 모든 스냅샷을 같은 길이로 맞추기
        min_length = min(len(snapshot) for snapshot in self.parameter_snapshots)
        aligned_snapshots = [snapshot[:min_length] for snapshot in self.parameter_snapshots]
        
        param_matrix = np.array(aligned_snapshots)
        
        if method == 'pca':
            reducer = PCA(n_components=n_components)
        elif method == 'tsne':
            reducer = TSNE(n_components=n_components, random_state=42)
        else:
            raise ValueError("지원되는 방법: 'pca', 'tsne'")
        
        reduced_params = reducer.fit_transform(param_matrix)
        
        self.pca = reducer  # 나중에 사용하기 위해 저장
        print("✅ 차원 축소 완료!")
        
        return reduced_params

print("🎨 GradientDescent3DVisualizer 클래스 정의 완료!")

## 🎬 3D 시각화 함수들

In [ ]:
def create_3d_loss_surface(alphas, betas, loss_surface, title="Loss Surface"):
    """3D Loss Surface 플롯 생성"""
    fig = go.Figure(data=[go.Surface(
        x=alphas,
        y=betas,
        z=loss_surface,
        colorscale='Viridis',
        opacity=0.8,
        name='Loss Surface'
    )])
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='Parameter Direction 1',
            yaxis_title='Parameter Direction 2',
            zaxis_title='Loss',
            camera=dict(eye=dict(x=1.2, y=1.2, z=1.2))
        ),
        width=800,
        height=600
    )
    
    return fig

def create_3d_gradient_path(trajectory, losses, title="Gradient Descent Path"):
    """3D Gradient Descent 경로 플롯"""
    if trajectory.shape[1] < 3:
        print("❌ 3D 시각화를 위해서는 최소 3차원이 필요합니다!")
        return None
    
    fig = go.Figure()
    
    # 경로 선
    fig.add_trace(go.Scatter3d(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        z=trajectory[:, 2],
        mode='lines+markers',
        line=dict(color='red', width=4),
        marker=dict(
            size=4,
            color=losses,
            colorscale='Plasma',
            colorbar=dict(title="Loss"),
            opacity=0.8
        ),
        name='Optimization Path'
    ))
    
    # 시작점
    fig.add_trace(go.Scatter3d(
        x=[trajectory[0, 0]],
        y=[trajectory[0, 1]],
        z=[trajectory[0, 2]],
        mode='markers',
        marker=dict(size=12, color='green', symbol='circle'),
        name='Start'
    ))
    
    # 끝점
    fig.add_trace(go.Scatter3d(
        x=[trajectory[-1, 0]],
        y=[trajectory[-1, 1]],
        z=[trajectory[-1, 2]],
        mode='markers',
        marker=dict(size=12, color='blue', symbol='circle'),
        name='End'
    ))
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='PC1',
            yaxis_title='PC2',
            zaxis_title='PC3',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        ),
        width=800,
        height=600
    )
    
    return fig

def create_training_dashboard(history):
    """훈련 과정 대시보드"""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Loss Curve', 'Gradient Norm', 'Parameter Norm', 'Learning Rate'),
        specs=[[{'secondary_y': False}, {'secondary_y': False}],
               [{'secondary_y': False}, {'secondary_y': False}]]
    )
    
    iterations = list(range(len(history['losses'])))
    
    # Loss curve
    fig.add_trace(
        go.Scatter(x=iterations, y=history['losses'], name='Loss', line=dict(color='blue')),
        row=1, col=1
    )
    
    # Gradient norm
    fig.add_trace(
        go.Scatter(x=iterations, y=history['gradient_norms'], name='Grad Norm', line=dict(color='red')),
        row=1, col=2
    )
    
    # Parameter norm
    fig.add_trace(
        go.Scatter(x=iterations, y=history['parameter_norms'], name='Param Norm', line=dict(color='green')),
        row=2, col=1
    )
    
    # Learning rate
    fig.add_trace(
        go.Scatter(x=iterations, y=history['learning_rates'], name='LR', line=dict(color='purple')),
        row=2, col=2
    )
    
    fig.update_layout(
        title="Training Metrics Dashboard",
        showlegend=False,
        height=600
    )
    
    return fig

def create_interactive_loss_landscape(visualizer, resolution=15):
    """Interactive Loss Landscape with Gradient Descent Path"""
    print("🎨 Interactive Loss Landscape 생성 중...")
    
    # 랜덤 방향 벡터 생성 (PCA 기준으로)
    total_params = sum(p.numel() for p in visualizer.model.parameters() if p.requires_grad)
    
    direction1 = torch.randn(total_params) * 0.1
    direction2 = torch.randn(total_params) * 0.1
    
    # 직교화
    direction2 = direction2 - torch.dot(direction1, direction2) / torch.dot(direction1, direction1) * direction1
    direction1 = direction1 / direction1.norm()
    direction2 = direction2 / direction2.norm()
    
    # 현재 파라미터를 중심으로 loss surface 계산
    center_params = torch.cat([p.data.view(-1) for p in visualizer.model.parameters() if p.requires_grad])
    
    alphas, betas, loss_surface = visualizer.compute_loss_surface_2d(
        center_params, direction1, direction2, 
        alpha_range=(-0.5, 0.5), beta_range=(-0.5, 0.5), 
        resolution=resolution
    )
    
    # Loss surface 플롯
    fig = create_3d_loss_surface(alphas, betas, loss_surface, "Interactive Loss Landscape")
    
    return fig

print("🎬 3D 시각화 함수들 정의 완료!")

## 🚀 모델 로드 및 훈련 시뮬레이션

In [ ]:
# 3D Visualizer 초기화
visualizer = GradientDescent3DVisualizer()

# Google Drive에서 모델 경로 설정
model_paths = [
    "/content/drive/MyDrive/hyperclova-deobfuscation-lora-with-10k-datasets",
    "/content/drive/MyDrive/hyperclova-deobfuscation-lora-with-30k-datasets"
]

available_model = None
for path in model_paths:
    if os.path.exists(path):
        available_model = path
        print(f"✅ Google Drive에서 모델 발견: {path}")
        break

if available_model:
    # Google Drive의 fine-tuned 모델 사용
    try:
        base_model_name = "ClovaAI/HyperCLOVA-X-SEED-Text-Instruct-0.5B"
        
        print(f"📥 베이스 모델 로드: {base_model_name}")
        visualizer.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            torch_dtype=torch.float16,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        
        print(f"🔧 Google Drive에서 LoRA 어댑터 로드: {available_model}")
        visualizer.model = PeftModel.from_pretrained(base_model, available_model)
        
        if visualizer.tokenizer.pad_token is None:
            visualizer.tokenizer.pad_token = visualizer.tokenizer.eos_token
            
        print("✅ Fine-tuned 모델 로드 완료!")
        
    except Exception as e:
        print(f"❌ Google Drive 모델 로드 실패: {str(e)}")
        print("🔄 기본 모델로 대체...")
        visualizer.load_model()
else:
    print("📥 Google Drive에서 모델을 찾을 수 없습니다. 기본 모델 로드...")
    visualizer.load_model()

# 샘플 데이터 준비
dataloader = visualizer.prepare_sample_data(num_samples=500)

In [ ]:
# 훈련 시뮬레이션 (짧은 훈련으로 gradient descent 추적)
print("🏃‍♂️ 훈련 시뮬레이션 시작...")

# 옵티마이저 설정
optimizer = optim.AdamW(visualizer.model.parameters(), lr=1e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

# 훈련 루프
visualizer.model.train()
num_epochs = 3
max_batches_per_epoch = 10

for epoch in range(num_epochs):
    print(f"\n📈 Epoch {epoch + 1}/{num_epochs}")
    
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(dataloader):
        if batch_idx >= max_batches_per_epoch:
            break
            
        # 데이터를 디바이스로 이동
        input_ids = input_ids.to(visualizer.device)
        attention_mask = attention_mask.to(visualizer.device)
        labels = labels.to(visualizer.device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = visualizer.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Gradient tracking
        visualizer.track_training_step(epoch, batch_idx, loss.item(), optimizer)
        
        # Optimizer step
        optimizer.step()
        
        if batch_idx % 5 == 0:
            print(f"  Batch {batch_idx}: Loss = {loss.item():.4f}")

print("\n✅ 훈련 시뮬레이션 완료!")
print(f"📊 수집된 데이터 포인트: {len(visualizer.training_history['losses'])}")

## 📊 3D 시각화 생성

In [ ]:
# 1. 훈련 메트릭 대시보드
print("📊 훈련 대시보드 생성 중...")
dashboard_fig = create_training_dashboard(visualizer.training_history)
dashboard_fig.show()

In [ ]:
# 2. 파라미터 궤적 3D 시각화
print("🔄 파라미터 궤적 차원 축소 중...")
reduced_trajectory = visualizer.reduce_dimensions(method='pca', n_components=3)

if reduced_trajectory is not None:
    print("🎨 3D 궤적 플롯 생성 중...")
    trajectory_fig = create_3d_gradient_path(
        reduced_trajectory, 
        visualizer.training_history['losses'],
        "3D Parameter Space Trajectory (PCA)"
    )
    trajectory_fig.show()
else:
    print("❌ 궤적 데이터가 충분하지 않습니다.")

In [ ]:
# 3. Interactive Loss Landscape
print("🗺️ Interactive Loss Landscape 생성 중...")
landscape_fig = create_interactive_loss_landscape(visualizer, resolution=12)
landscape_fig.show()

## 🔍 모델 비교 분석

In [ ]:
def compare_multiple_models():
    """여러 모델의 훈련 과정 비교"""
    model_configs = [
        {'lr': 1e-4, 'weight_decay': 0.01, 'name': 'High LR'},
        {'lr': 1e-5, 'weight_decay': 0.01, 'name': 'Medium LR'},
        {'lr': 1e-6, 'weight_decay': 0.01, 'name': 'Low LR'}
    ]
    
    comparison_data = {}
    
    print("🔍 여러 설정으로 모델 비교 중...")
    
    for config in model_configs:
        print(f"\n⚙️ 설정: {config['name']} (LR: {config['lr']})")
        
        # 새로운 visualizer 생성
        temp_visualizer = GradientDescent3DVisualizer()
        temp_visualizer.model = visualizer.model  # 같은 모델 사용
        temp_visualizer.tokenizer = visualizer.tokenizer
        
        # 옵티마이저 설정
        temp_optimizer = optim.AdamW(
            temp_visualizer.model.parameters(), 
            lr=config['lr'], 
            weight_decay=config['weight_decay']
        )
        
        # 짧은 훈련
        temp_visualizer.model.train()
        for batch_idx, (input_ids, attention_mask, labels) in enumerate(dataloader):
            if batch_idx >= 5:  # 빠른 비교를 위해 5배치만
                break
                
            input_ids = input_ids.to(temp_visualizer.device)
            attention_mask = attention_mask.to(temp_visualizer.device)
            labels = labels.to(temp_visualizer.device)
            
            temp_optimizer.zero_grad()
            outputs = temp_visualizer.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            
            temp_visualizer.track_training_step(0, batch_idx, loss.item(), temp_optimizer)
            temp_optimizer.step()
        
        comparison_data[config['name']] = temp_visualizer.training_history
    
    # 비교 플롯 생성
    fig = go.Figure()
    
    for name, history in comparison_data.items():
        fig.add_trace(go.Scatter(
            x=list(range(len(history['losses']))),
            y=history['losses'],
            mode='lines+markers',
            name=f'{name} - Loss',
            line=dict(width=3)
        ))
    
    fig.update_layout(
        title="Model Configuration Comparison",
        xaxis_title="Training Step",
        yaxis_title="Loss",
        width=800,
        height=500
    )
    
    return fig

# 모델 비교 실행
comparison_fig = compare_multiple_models()
comparison_fig.show()

In [ ]:
# Google Drive에 저장된 두 모델 비교 (10k vs 30k)
def compare_drive_models():
    """Google Drive의 10k와 30k 모델 비교"""
    model_configs = [
        {
            'path': '/content/drive/MyDrive/hyperclova-deobfuscation-lora-with-10k-datasets',
            'name': '10K Dataset Model',
            'color': 'blue'
        },
        {
            'path': '/content/drive/MyDrive/hyperclova-deobfuscation-lora-with-30k-datasets', 
            'name': '30K Dataset Model',
            'color': 'red'
        }
    ]
    
    comparison_data = {}
    base_model_name = "ClovaAI/HyperCLOVA-X-SEED-Text-Instruct-0.5B"
    
    print("🔍 Google Drive의 두 모델 비교 중...")
    
    for config in model_configs:
        if not os.path.exists(config['path']):
            print(f"❌ 모델을 찾을 수 없습니다: {config['path']}")
            continue
            
        print(f"\n⚙️ 모델 로드: {config['name']}")
        
        try:
            # 베이스 모델 로드
            tokenizer = AutoTokenizer.from_pretrained(base_model_name)
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
                torch_dtype=torch.float16,
                device_map="auto" if torch.cuda.is_available() else None,
                trust_remote_code=True
            )
            
            # LoRA 어댑터 적용
            model = PeftModel.from_pretrained(base_model, config['path'])
            
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token
            
            # 새로운 visualizer 생성
            temp_visualizer = GradientDescent3DVisualizer()
            temp_visualizer.model = model
            temp_visualizer.tokenizer = tokenizer
            
            # 옵티마이저 설정
            temp_optimizer = optim.AdamW(temp_visualizer.model.parameters(), lr=1e-5, weight_decay=0.01)
            
            # 짧은 훈련으로 성능 비교
            temp_visualizer.model.train()
            sample_data = temp_visualizer.prepare_sample_data(num_samples=200)
            
            for batch_idx, (input_ids, attention_mask, labels) in enumerate(sample_data):
                if batch_idx >= 8:  # 빠른 비교를 위해 8배치만
                    break
                    
                input_ids = input_ids.to(temp_visualizer.device)
                attention_mask = attention_mask.to(temp_visualizer.device)
                labels = labels.to(temp_visualizer.device)
                
                temp_optimizer.zero_grad()
                outputs = temp_visualizer.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()
                
                temp_visualizer.track_training_step(0, batch_idx, loss.item(), temp_optimizer)
                temp_optimizer.step()
            
            comparison_data[config['name']] = {
                'history': temp_visualizer.training_history,
                'color': config['color']
            }
            
            print(f"✅ {config['name']} 분석 완료")
            
        except Exception as e:
            print(f"❌ {config['name']} 로드 실패: {str(e)}")
            continue
    
    # 비교 플롯 생성
    if comparison_data:
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Loss Comparison', 'Gradient Norm', 'Parameter Norm', 'Convergence Rate'),
            specs=[[{'secondary_y': False}, {'secondary_y': False}],
                   [{'secondary_y': False}, {'secondary_y': False}]]
        )
        
        for name, data in comparison_data.items():
            history = data['history']
            color = data['color']
            steps = list(range(len(history['losses'])))
            
            # Loss 비교
            fig.add_trace(
                go.Scatter(x=steps, y=history['losses'], name=f'{name} - Loss', 
                         line=dict(color=color, width=3)), row=1, col=1
            )
            
            # Gradient Norm 비교
            fig.add_trace(
                go.Scatter(x=steps, y=history['gradient_norms'], name=f'{name} - Grad Norm',
                         line=dict(color=color, dash='dash')), row=1, col=2
            )
            
            # Parameter Norm 비교
            fig.add_trace(
                go.Scatter(x=steps, y=history['parameter_norms'], name=f'{name} - Param Norm',
                         line=dict(color=color, dash='dot')), row=2, col=1
            )
            
            # 수렴률 계산 (loss 감소 속도)
            if len(history['losses']) > 1:
                convergence_rate = []
                for i in range(1, len(history['losses'])):
                    rate = (history['losses'][i-1] - history['losses'][i]) / history['losses'][i-1]
                    convergence_rate.append(rate)
                
                fig.add_trace(
                    go.Scatter(x=list(range(1, len(history['losses']))), y=convergence_rate, 
                             name=f'{name} - Conv Rate', line=dict(color=color, dash='dashdot')), 
                    row=2, col=2
                )
        
        fig.update_layout(
            title="10K vs 30K Dataset Model Comparison",
            height=800,
            showlegend=True
        )
        
        return fig
    else:
        print("❌ 비교할 수 있는 모델이 없습니다.")
        return None

# Google Drive 모델 비교 실행
drive_comparison_fig = compare_drive_models()
if drive_comparison_fig:
    drive_comparison_fig.show()

## 💾 결과 저장 및 분석

In [ ]:
# 훈련 히스토리를 DataFrame으로 변환
history_df = pd.DataFrame({
    'step': range(len(visualizer.training_history['losses'])),
    'epoch': visualizer.training_history['epochs'],
    'batch_idx': visualizer.training_history['batch_indices'],
    'loss': visualizer.training_history['losses'],
    'gradient_norm': visualizer.training_history['gradient_norms'],
    'parameter_norm': visualizer.training_history['parameter_norms'],
    'learning_rate': visualizer.training_history['learning_rates']
})

print("📊 훈련 히스토리 통계:")
print(history_df.describe())

# CSV로 저장
output_path = "/Users/jw/PycharmProjects/FineTuningLLM/gradient_descent_analysis.csv"
history_df.to_csv(output_path, index=False)
print(f"💾 결과 저장 완료: {output_path}")

# 최종 분석 리포트
print(f"""
🎯 Gradient Descent 분석 결과:

📈 훈련 개요:
- 총 훈련 스텝: {len(visualizer.training_history['losses'])}
- 최종 Loss: {visualizer.training_history['losses'][-1]:.6f}
- 초기 Loss: {visualizer.training_history['losses'][0]:.6f}
- Loss 감소율: {(visualizer.training_history['losses'][0] - visualizer.training_history['losses'][-1]) / visualizer.training_history['losses'][0] * 100:.2f}%

🔍 Gradient 분석:
- 평균 Gradient Norm: {np.mean(visualizer.training_history['gradient_norms']):.6f}
- 최대 Gradient Norm: {np.max(visualizer.training_history['gradient_norms']):.6f}
- 최소 Gradient Norm: {np.min(visualizer.training_history['gradient_norms']):.6f}

⚙️ 파라미터 분석:
- 평균 Parameter Norm: {np.mean(visualizer.training_history['parameter_norms']):.6f}
- 파라미터 변화량: {abs(visualizer.training_history['parameter_norms'][-1] - visualizer.training_history['parameter_norms'][0]):.6f}
""")

In [ ]:
# Google Colab GPU 메모리 관리
def optimize_gpu_memory():
    """GPU 메모리 최적화"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"🔧 GPU 메모리 정리 완료")
        print(f"💾 현재 GPU 메모리 사용량: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        print(f"💾 최대 GPU 메모리 사용량: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
    else:
        print("CPU 모드에서 실행 중입니다.")

# 메모리 최적화 실행
optimize_gpu_memory()

# Colab에서 실행 시간 측정
import time
start_time = time.time()

print(f"\n⏱️ 총 실행 시간: {time.time() - start_time:.2f}초")
print("\n🎉 3D Gradient Descent 시각화 완료!")
print("\n📝 사용법:")
print("1. 위의 시각화 결과를 통해 모델의 최적화 과정을 분석하세요")
print("2. Google Drive에 저장된 HTML 파일을 다운로드하여 오프라인에서도 확인 가능합니다")
print("3. CSV 데이터를 통해 추가적인 분석을 수행하세요")

In [ ]:
# Google Drive에 결과 저장
output_dir = "/content/drive/MyDrive/gradient_descent_analysis/"
os.makedirs(output_dir, exist_ok=True)

# 훈련 히스토리를 DataFrame으로 변환
history_df = pd.DataFrame({
    'step': range(len(visualizer.training_history['losses'])),
    'epoch': visualizer.training_history['epochs'],
    'batch_idx': visualizer.training_history['batch_indices'],
    'loss': visualizer.training_history['losses'],
    'gradient_norm': visualizer.training_history['gradient_norms'],
    'parameter_norm': visualizer.training_history['parameter_norms'],
    'learning_rate': visualizer.training_history['learning_rates']
})

print("📊 훈련 히스토리 통계:")
print(history_df.describe())

# Google Drive에 CSV로 저장
output_path = os.path.join(output_dir, "gradient_descent_analysis.csv")
history_df.to_csv(output_path, index=False)
print(f"💾 결과 저장 완료: {output_path}")

# 시각화 결과도 HTML로 저장
if 'dashboard_fig' in locals():
    dashboard_fig.write_html(os.path.join(output_dir, "training_dashboard.html"))
    print("📊 대시보드 HTML 저장 완료")

if 'trajectory_fig' in locals():
    trajectory_fig.write_html(os.path.join(output_dir, "3d_trajectory.html"))
    print("🎨 3D 궤적 HTML 저장 완료")

if 'landscape_fig' in locals():
    landscape_fig.write_html(os.path.join(output_dir, "loss_landscape.html"))
    print("🗺️ Loss Landscape HTML 저장 완료")

# 최종 분석 리포트
print(f"""
🎯 Gradient Descent 분석 결과:

📈 훈련 개요:
- 총 훈련 스텝: {len(visualizer.training_history['losses'])}
- 최종 Loss: {visualizer.training_history['losses'][-1]:.6f}
- 초기 Loss: {visualizer.training_history['losses'][0]:.6f}
- Loss 감소율: {(visualizer.training_history['losses'][0] - visualizer.training_history['losses'][-1]) / visualizer.training_history['losses'][0] * 100:.2f}%

🔍 Gradient 분석:
- 평균 Gradient Norm: {np.mean(visualizer.training_history['gradient_norms']):.6f}
- 최대 Gradient Norm: {np.max(visualizer.training_history['gradient_norms']):.6f}
- 최소 Gradient Norm: {np.min(visualizer.training_history['gradient_norms']):.6f}

⚙️ 파라미터 분석:
- 평균 Parameter Norm: {np.mean(visualizer.training_history['parameter_norms']):.6f}
- 파라미터 변화량: {abs(visualizer.training_history['parameter_norms'][-1] - visualizer.training_history['parameter_norms'][0]):.6f}

💾 저장된 파일:
- CSV 데이터: {output_path}
- HTML 시각화: {output_dir}
""")

## 🎉 결론 및 인사이트

### 📊 주요 발견사항:

1. **Loss Landscape**: 3D 시각화를 통해 모델이 최적화되는 과정을 직관적으로 확인
2. **Gradient Descent Path**: 파라미터 공간에서의 최적화 경로를 3차원으로 추적
3. **수렴 패턴**: 다양한 하이퍼파라미터 설정에 따른 수렴 속도와 안정성 비교

### 🔍 활용 방안:

- **하이퍼파라미터 튜닝**: Loss landscape를 통한 최적 학습률 찾기
- **모델 안정성 분석**: Gradient norm 변화를 통한 훈련 안정성 평가  
- **수렴 모니터링**: 실시간 3D 시각화를 통한 훈련 과정 모니터링

### 📈 향후 개선점:

- 더 긴 훈련 과정에서의 long-term 패턴 분석
- 다양한 데이터셋에 대한 일반화 성능 비교
- Interactive dashboard를 통한 실시간 하이퍼파라미터 조정

이 노트북을 통해 fine-tuning 과정의 gradient descent를 3D로 시각화하여 모델 최적화 과정을 더 잘 이해할 수 있습니다! 🚀